# 수열과 쿼리 17

## 문제

* 길이가 N인 수열 A1, A2, ..., AN이 주어진다. 이때, 다음 쿼리를 수행하는 프로그램을 작성하시오.
    * 1 i v : Ai를 v로 바꾼다. (1 ≤ i ≤ N, 1 ≤ v ≤ 109)
    * 2 i j : Ai, Ai+1, ..., Aj에서 크기가 가장 작은 값을 출력한다. (1 ≤ i ≤ j ≤ N)

* 수열의 인덱스는 1부터 시작한다.

## 입력

* 첫째 줄에 수열의 크기 N이 주어진다. (1 ≤ N ≤ 100,000)

* 둘째 줄에는 A1, A2, ..., AN이 주어진다. (1 ≤ Ai ≤ 10^9)

* 셋째 줄에는 쿼리의 개수 M이 주어진다. (1 ≤ M ≤ 100,000)

* 넷째 줄부터 M개의 줄에는 쿼리가 주어진다.

## 출력

* 2번 쿼리에 대해서 정답을 한 줄에 하나씩 순서대로 출력한다.

## Code

In [1]:
# 14438_수열과 쿼리17

import sys

submit = False
input = sys.stdin.readline if submit else input

def solution():
    N = int(input())
    numbers = list(map(int, input().split()))
    M = int(input())

    for _ in range(M):
        cmd, i, v = map(int, input().split())
        if cmd == 1:
            numbers[i-1] = v
        elif cmd == 2:
            i = i-1 if i-1 >= 0 else 0
            print(min(numbers[i:v]))

In [ ]:
# 14438_수열과 쿼리17

import sys
import math

submit = False
input = sys.stdin.readline if submit else input

def makeSegTree(arr, seg, idx, st, ed):
    if st == ed:
        seg[idx] = arr[st]
        return seg[idx]
    
    mid = (st + ed) // 2
    left, right = makeSegTree(arr, seg, idx * 2, st, mid), makeSegTree(arr, seg, idx * 2 + 1, mid+1, ed)
    seg[idx] = min(left,  right)
    return seg[idx]

def find(seg, range1, range2, idx, st, ed):
    if range2 < st or range1 > ed:
        return 10**9+1
    if range1 <= st and ed <= range2:
        return seg[idx]
    
    mid = (st + ed) // 2
    left, right = find(seg, range1, range2, idx * 2, st, mid), find(seg, range1, range2, idx * 2 + 1, mid + 1, ed)
    return min(left, right)

def update(seg, st, ed, idx, update_idx, update_num):
    if (st > update_idx) or (ed < update_idx):
        return

    if st == ed:
        seg[idx] = update_num    
        return

    mid = (st + ed) // 2
    update(seg, st, mid, idx * 2, update_idx, update_num)
    update(seg, mid + 1, ed, idx * 2 + 1, update_idx, update_num)
    seg[idx] = min(seg[idx*2], seg[idx*2+1])

def solution():
    N = int(input())
    numbers = list(map(int, input().split()))
    M = int(input())

    h = math.ceil(math.log2(N)) + 1
    nodeNum = 1 << h
    seg = [0 for _ in range(nodeNum)]
    makeSegTree(numbers, seg, 1, 0, len(numbers) - 1)

    for _ in range(M):
        cmd, i, v = map(int, input().split())
        if cmd == 1:
            numbers[i-1] = v
            update(seg, 1, N, 1, i, v)

        elif cmd == 2:
            print(find(seg, i, v, 1, 1, N))

## 예제입력 - 출력

In [2]:
solution()

3
2
2
3


## Note

### 풀이 1

* 1. indexing을 사용해서 a가 1이면 숫자를 입력받은 List(numbers)의 숫자를 변경하고 2면 구간의 최소값을 출력한다.

### 풀이 2

* 1. Segment Tree를 이용하여 구간 별 최소값을 미리 저장한다.
    * 1\) 전체 구간을 반으로 나눠 구간에 존재하는 숫자가 1개만 남을때까지 구간을 나눈다.
    * 2\) 존재하는 숫자가 1개만 남으면 해당 Node의 값은 자기 자신이므로 결과를 list(seg)에 저장한다.
        * 저장할 때, 상위 index의 *2와 *2+1에 저장한다.
        * 1 -> 2, 3 / 2 -> 4, 5 / 3 -> 6, 7 ...
        * eg. numbers: [5, 4, 3, 2, 1]
        * seg index: 1 -> 2, 3 / 2 -> 4, 5 / 3 -> 6, 7 ...
            * seg index 1: 1, 전체 구간의 최소값
            * \# ------------------------------- # 
            * seg index 2: 3, index 0 ~ 2의 최소값
            * seg index 3: 1, index 3 ~ 4의 최소값
            * \# ------------------------------- #
            * seg index 4: 4, index 0 ~ 1의 최소값
            * seg index 5: 3, index 2의 값
            * seg index 6: 2, index 3의 값
            * seg index 7: 1, index 4의 값
            * \# ------------------------------- #
            * seg index 8: 5, index 0의 값
            * seg index 9: 4, index 1의 값

* 2. 숫자가 변경되면 Segment Tree를 탐색하면서 구간에 포함되는 Node들의 숫자(구간의 합)을 변경한다.
    * 1\) 재귀 함수를 통해 가장 깊이가 깊은(구간의 길이가 0인 자기 자신의 Node) 구간의 수를 먼저 변경한다.
    * 2\) seg에서 나머지 구간의 최소값을 갱신한다.
    * eg. 
    * numbers: [5, 4, 3, 2, 1]
    * seg: [0, 1, 3, 1, 4, 3, 2, 1, 5, 4, 0, 0, 0, 0, 0, 0]
    * \# ------------------------------- #
    * numbers: [5, 4, 3, 2, 3]
    * seg: [0, 2, 3, 2, 4, 3, 2, 3, 5, 4, 0, 0, 0, 0, 0, 0]

* 3. 주어진 구간에 따라 Segment Tree를 탐색하면서 해당하는 구간의 최소값을 찾는다.
    * eg. numbers: [5, 4, 3, 2, 1]
    * seg: [0, 1, 3, 1, 4, 3, 2, 1, 5, 4, 0, 0, 0, 0, 0, 0]
    * st: 1, ed: 5, range1: 1, range2: 3
    * OUT | st: 1 ed: 5 -> 주어진 구간이 값을 찾을 구간보다 크므로 찾을 구간(st~ed)을 (1~3), (4~5) 두 구간으로 나눈다
    *  IN | st: 1 ed: 3 -> 찾을 구간(st: 1 ~ ed: 3)이 주어진 구간(range_1: 1 ~ range_2: 3)에 포함되므로  최소값인 3을 반환한다.
    * OUT | st: 4 ed: 5 -> 찾을 구간인 (st: 4 ~ ed:5)가 주어진 구간인 (range_1: 1 ~ range_2: 3)을 벗어나 매우 큰 값(10**9+1)을 반환한다.
    * 3과 10**9+1 중 최소값인 3을 반환한다.

https://www.acmicpc.net/problem/14438